In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import math

# Define Tree Geometry
def get_tree_polygon(x=0, y=0, angle=0):
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    coords = [
        (0.0, tip_y),
        (top_w/2, tier_1_y), (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y), (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y), (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y), (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y), (-top_w/2, tier_1_y)
    ]
    
    poly = Polygon(coords)
    if angle != 0:
        poly = affinity.rotate(poly, angle, origin=(0, 0))
    if x != 0 or y != 0:
        poly = affinity.translate(poly, xoff=x, yoff=y)
    return poly

tree_area = get_tree_polygon().area
print(f"Tree Area: {tree_area}")


In [ ]:
# Function to check overlap for a given lattice
def check_lattice_overlap(v1, v2, trees_in_unit_cell):
    """
    Checks if trees in the unit cell overlap with neighbors in the lattice.
    v1, v2: Lattice vectors (tuples)
    trees_in_unit_cell: list of shapely Polygons
    
    Returns: True if overlap detected, False otherwise
    """
    # Check self-overlap (within unit cell)
    for i in range(len(trees_in_unit_cell)):
        for j in range(i + 1, len(trees_in_unit_cell)):
            if trees_in_unit_cell[i].intersects(trees_in_unit_cell[j]):
                return True

    # Check neighbor overlap
    # We need to check enough neighbors. For a dense packing, checking immediate neighbors (radius 1 or 2) is usually enough.
    search_radius = 2
    
    for i in range(-search_radius, search_radius + 1):
        for j in range(-search_radius, search_radius + 1):
            if i == 0 and j == 0:
                continue
                
            dx = i * v1[0] + j * v2[0]
            dy = i * v1[1] + j * v2[1]
            
            for tree_base in trees_in_unit_cell:
                # Translate tree by lattice vector
                tree_shifted = affinity.translate(tree_base, xoff=dx, yoff=dy)
                
                for tree_target in trees_in_unit_cell:
                    if tree_shifted.intersects(tree_target):
                        # Use a small buffer for numerical stability if needed, but shapely is usually fine
                        # Intersection area check might be more robust than simple boolean intersects
                        if tree_shifted.intersection(tree_target).area > 1e-6:
                            return True
    return False

# Visualization
def plot_lattice(v1, v2, trees_in_unit_cell, n_repeats=3):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    for i in range(-n_repeats, n_repeats + 1):
        for j in range(-n_repeats, n_repeats + 1):
            dx = i * v1[0] + j * v2[0]
            dy = i * v1[1] + j * v2[1]
            
            for tree in trees_in_unit_cell:
                tree_shifted = affinity.translate(tree, xoff=dx, yoff=dy)
                x, y = tree_shifted.exterior.xy
                ax.fill(x, y, alpha=0.5, edgecolor='black')
                
    ax.set_aspect('equal')
    plt.show()


In [ ]:
# Optimization Strategy:
# We will use a simple grid search or optimization loop to find the best lattice vectors.
# Since we suspect 0 and 180 degrees are optimal, we focus on that.

# Scenario 1: Single Lattice (All 0 degrees)
# Lattice vectors: v1 = (dx, 0), v2 = (shift_x, dy)
# We want to minimize dx * dy.

def optimize_single_lattice_0deg():
    best_area = float('inf')
    best_params = None
    
    # Coarse grid search
    # dx must be at least width of tree approx 0.7
    # dy must be at least height of tree approx 1.0
    
    # Actually, they can interlock, so dy can be smaller if shift_x is used.
    
    # Let's try to find the "kissing" distance.
    # Fix y-spacing, find min x-spacing.
    
    t0 = get_tree_polygon(0, 0, 0)
    
    # Grid search ranges
    dy_range = np.linspace(0.2, 1.1, 50) # Height is 1.0 (0.8 - -0.2)
    shift_range = np.linspace(-0.4, 0.4, 20)
    
    for dy in dy_range:
        for shift in shift_range:
            # Binary search for min dx
            low = 0.1
            high = 1.0
            min_dx = high
            
            # Check if valid at high
            v1 = (high, 0)
            v2 = (shift, dy)
            if check_lattice_overlap(v1, v2, [t0]):
                continue # Even at max spacing it overlaps? Unlikely.
            
            for _ in range(10):
                mid = (low + high) / 2
                v1 = (mid, 0)
                v2 = (shift, dy)
                if check_lattice_overlap(v1, v2, [t0]):
                    low = mid
                else:
                    min_dx = mid
                    high = mid
            
            # Calculate area
            # Area of parallelogram = base * height = dx * dy
            area = min_dx * dy
            if area < best_area:
                best_area = area
                best_params = (min_dx, dy, shift)
                
    return best_area, best_params

best_area_single, params_single = optimize_single_lattice_0deg()
print(f"Best Single Lattice Area: {best_area_single}")
print(f"Params (dx, dy, shift): {params_single}")
print(f"Density: {tree_area / best_area_single}")

t0 = get_tree_polygon(0, 0, 0)
v1 = (params_single[0], 0)
v2 = (params_single[2], params_single[1])
plot_lattice(v1, v2, [t0])


In [ ]:
# Scenario 2: Double Lattice (0 and 180 degrees)
# Unit cell contains T1(0,0,0) and T2(x2, y2, 180)
# Lattice vectors v1=(dx, 0), v2=(0, dy) -- Simple rectangular lattice of unit cells
# Or more general v1, v2.
# Let's assume a rectangular grid of the PAIR is good enough, or a hexagonal arrangement of pairs.
# Actually, the most dense packing of triangles is usually alternating rows.
# Row 1: 0 deg, spacing dx.
# Row 2: 180 deg, spacing dx. Shifted by (sx, sy).
# This forms a lattice with 2 trees basis.

def optimize_double_lattice():
    best_area_per_tree = float('inf')
    best_params = None
    
    t1 = get_tree_polygon(0, 0, 0)
    t2 = get_tree_polygon(0, 0, 180) # Centered at 0,0 initially
    
    # We need to place t2 relative to t1 at (rx, ry)
    # And define lattice vectors v1, v2.
    # Simplified model:
    # Rows of alternating trees?
    # Or Row 1 = all 0, Row 2 = all 180.
    
    # Let's try: Row of 0s, Row of 180s, Row of 0s...
    # Horizontal spacing dx.
    # Vertical spacing dy between rows.
    # Shift sx between rows.
    # This is effectively a single lattice if we consider the unit cell to be one 0 and one 180?
    # No, if row 1 is 0, row 2 is 180, row 3 is 0...
    # Then v1 = (dx, 0). v2 = (sx, dy).
    # Basis: T1 at (0,0), T2 at (rx, ry).
    # If it's a regular pattern, T2 is likely at (sx/2, dy/2) relative to T1?
    # Let's search for relative position of T2 (rx, ry) and lattice parameters.
    
    # To reduce dimensions:
    # Assume T2 is "locked" into T1 in a dense configuration.
    # Let's just search for dense T1-T2 pair first? No, they interact with neighbors.
    
    # Let's try a specific pattern:
    # T1 at (0,0). T2 at (dx/2, y_offset).
    # v1 = (dx, 0).
    # v2 = (0, dy_double). (Rectangular unit cell of size dx * dy_double containing 2 trees)
    # This implies columns of 0s and columns of 180s.
    
    # Alternative: Rows.
    # Row 0: 0 deg. Spacing dx.
    # Row 1: 180 deg. Spacing dx. Offset (sx, sy) from Row 0.
    # Row 2: 0 deg. Spacing dx. Offset (sx, sy) from Row 1? No, usually back to start or 2*offset.
    # Let's assume periodicity v2 = (0, 2*sy) ?
    
    # Let's stick to the definition:
    # Unit cell: T1(0), T2(180) at (rx, ry).
    # Lattice: v1=(dx, 0), v2=(sx, sy).
    # Minimize |v1 x v2| / 2.
    
    # Grid search is too expensive for 5 params (dx, sx, sy, rx, ry).
    # Heuristic:
    # 1. Place T1 and T2 as close as possible (optimize rx, ry for just 2 trees).
    # 2. Use that bounding box to guess lattice?
    
    # Better: Use scipy.optimize.minimize
    from scipy.optimize import minimize
    
    def objective(params):
        dx, sx, sy, rx, ry = params
        # Constraints: dx > 0, dy > 0 (implied by area calc)
        # Area of unit cell = dx * sy (assuming v1 aligned with x)
        # Wait, v1=(dx, 0), v2=(sx, sy). Area = dx * sy.
        # We want to minimize area.
        return dx * abs(sy)

    def constraint_overlap(params):
        dx, sx, sy, rx, ry = params
        if dx <= 0 or sy <= 0: return -1.0
        
        v1 = (dx, 0)
        v2 = (sx, sy)
        
        t1_base = get_tree_polygon(0, 0, 0)
        t2_base = get_tree_polygon(rx, ry, 180)
        
        if check_lattice_overlap(v1, v2, [t1_base, t2_base]):
            return -1.0 # Violation
        return 1.0 # OK

    # This constraint is hard for optimizers.
    # Let's go back to grid search with assumptions.
    # Assumption: T2 is centered horizontally between T1s? rx = dx/2?
    # Assumption: Rectangular lattice? sx = 0?
    
    # Let's try "Alternating Rows" pattern specifically.
    # v1 = (dx, 0).
    # T1 at (0,0).
    # T2 at (shift_x, shift_y).
    # v2 = (0, 2*shift_y)? No.
    
    # Let's try to pack just 0 and 180 pairs in a strip, then stack strips.
    pass

# Let's run the single lattice optimization first to get a baseline.
